In [1]:
import requests,time,os

In [3]:
url="https://www.carwale.com/new-cars/"
headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0"
}

response=requests.get(url,headers=headers)
response.encoding='utf-8'

file_name="Carwaale_Newcars.html"
with open(file_name,"w",encoding='utf-8') as f:
    f.write(response.text)
print("✅ Saved raw HTML as carwale_new_cars.html")

✅ Saved raw HTML as carwale_new_cars.html


In [7]:
from bs4 import BeautifulSoup
import csv
with open(file_name,"r",encoding='utf-8') as f:
    htmls=f.read()

soup=BeautifulSoup(htmls,"html.parser")
print(soup.prettify())


<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb#">
 <head>
  <meta charset="utf-8"/>
  <meta content="#0e3a50" name="theme-color"/>
  <meta content="max-snippet:-1, max-image-preview:large" name="robots">
   <title data-react-helmet="true" itemprop="name">
    New Cars in India 2025 | New Car Information | Best New Cars - CarWale
   </title>
   <link crossorigin="" href="https://imgd.aeplcdn.com" rel="preconnect dns-prefetch"/>
   <link crossorigin="" href="https://stc.aeplcdn.com" rel="preconnect dns-prefetch"/>
   <link crossorigin="" href="https://www.googletagmanager.com" rel="preconnect dns-prefetch"/>
   <link crossorigin="" href="https://www.googletagservices.com" rel="preconnect dns-prefetch"/>
   <link crossorigin="" href="https://bid.g.doubleclick.net" rel="preconnect dns-prefetch"/>
   <link crossorigin="" href="https://pagead2.googlesyndication.com" rel="dns-prefetch"/>
   <link crossori

This code scrapes the brands section from the CarWale New Cars page.
It goes through all the car brands listed, extracts the brand name and the brand page link, and saves them into a CSV file named carwale_brands.csv.

The CSV has two columns:

Brand Name – e.g., Maruti Suzuki, Tata, Hyundai

Brand Link – the URL to the page where all cars of that brand are listed

The tag i.e section ,inside that, the id and then the 'a' tag inside, then 'title' and 'href' but only those href that ends with cars as brands href ends with cars?

In [19]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv

# Load saved HTML file
with open("Carwaale_Newcars.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# Select brand links
brand_anchors = soup.select('section[data-section-id="all-brands-section"] a[title][href$="-cars/"]')

brands = []
base = "https://www.carwale.com"

for a in brand_anchors:
    name = a.get("title", "").strip()
    link = urljoin(base, a.get("href", "").strip())
    if name and link:
        brands.append([name, link])

# Save to CSV
with open("carwale_brands.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Brand Name", "Brand Link"])  # Header
    writer.writerows(brands)

print("✅ Saved brand details to carwale_brands.csv")


✅ Saved brand details to carwale_brands.csv


This script extracts detailed information about all new cars listed on CarWale.
It parses the saved HTML file, looks for car detail links, and collects each car’s name, price, and direct link. To keep the data clean, duplicate entries are removed. Finally, the extracted information is saved into a CSV file (carwale_new_cars.csv) with three columns: Car Name, Price, and Link

In [21]:
import re
import csv
from bs4 import BeautifulSoup

# Load saved HTML file
with open("Carwaale_Newcars.html", "r", encoding="utf-8") as f:
    html = f.read()

soup = BeautifulSoup(html, "html.parser")

# Step 2: Extract cars
cars = []
seen_links = set()  # to avoid duplicates

for a in soup.select('a[href*="-cars/"]'):
    href = a.get("href", "")
    
    # Match only car detail pages (brand-cars/model)
    if re.search(r"/[a-z0-9-]+-cars/[a-z0-9-]+/?$", href):
        link = "https://www.carwale.com" + href

        # Car name
        name = (a.get("title") or a.get_text(" ", strip=True)).strip()

        # Price (search in parent text)
        card = a.find_parent()
        price = ""
        if card:
            txt = card.get_text(" ", strip=True)
            m = re.search(r"Rs\.\s*[\d\.,]+(?:\s*Lakh|\s*Crore)?", txt)
            if m:
                price = m.group(0)

        # Save if price exists and link not duplicate
        if price and link not in seen_links:
            cars.append([name, price, link])
            seen_links.add(link)

# Step 3: Save to CSV
csv_file = "carwale_new_cars.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Car Name", "Price", "Link"])
    writer.writerows(cars)

print(f"✅ Extracted {len(cars)} unique cars and saved to {csv_file}")


✅ Extracted 43 unique cars and saved to carwale_new_cars.csv
